In [ ]:
import processing as p
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
paths = ["problems_2023_01_30/problems MoonBoard 2016 .json", 
         "problems_2023_01_30/problems MoonBoard Masters 2017 40.json",
         "problems_2023_01_30/problems MoonBoard Masters 2019 40.json"]
years = [2016, 2017, 2019]

In [ ]:
def plot_sandbagging(df, year):
    # vizualize the number of boulders upgraded and downgraded per setter (df.setby)
    up_down_df = (df
        .groupby('setby')
        .agg({'upgraded': 'sum', 'downgraded': 'sum', 'name':'count'})
        .sort_values('name', ascending=False)
        .head(30)
    )
    up_down_df = (up_down_df
                [(up_down_df.upgraded > 0) | (up_down_df.downgraded > 0)] 
    )
    up_down_df.head(20)

        # Seaborn settings
    sns.set_theme(style="whitegrid")

    # Bubble chart
    plt.figure(figsize=(15, 10))

    # Define size range for the bubbles
    size_range = (20, 2000)

    # Plotting upgraded bubbles in blue
    sns.scatterplot(
        x=up_down_df.index, y=up_down_df['name'],
        size=up_down_df['upgraded'], sizes=size_range,
        hue=up_down_df['upgraded'], palette="Blues",
        alpha=0.8
    )

    # Plotting downgraded bubbles in red on top of upgraded
    sns.scatterplot(
        x=up_down_df.index, y=up_down_df['name'],
        size=up_down_df['downgraded'], sizes=size_range,
        hue=up_down_df['downgraded'], palette="Reds",
        alpha=0.8
    )

    plt.ylabel('Number of Routes/Problems Set')
    plt.title(f"The biggest sandbaggers and softies on the Moonboard - {year} set")
    plt.xticks(rotation=45)

    plt.tight_layout(rect=[0, 0, 0.85, 1]) 
    plt.show()


In [ ]:
for year,path in zip(years, paths):
    df = p.get_df(path)
    df = p.group_by_problem(df)
    df = p.prepare_features(df, benchmark=True)
    plot_sandbagging(df, year)

In [ ]:
for year,path in zip(years, paths):
    df = p.get_df(path)
    df = p.group_by_problem(df)
    df = p.prepare_features(df, benchmark=False)
    # show the average grade per year
    sns.barplot(x='year', y='grade', data=df, alpha=0.8, label=year)
    # for the y label re-translate to the original grade
    plt.yticks(np.arange(0, 8, 1), [p.GRADE_DICT_REVERSE[x] for x in np.arange(0, 8, 1)])
    plt.title('Average grade per year and set')

In [ ]:
df = p.get_df(paths[1])
df = p.group_by_problem(df)
df = p.prepare_features(df, benchmark=True)

In [ ]:
def get_holds_heatmap(df, kpi='grade'):
    horizontal = list("ABCDEFGHIJK")
    vertical = list(range(1, 19))
    heatmap_df = pd.DataFrame(index=vertical, columns=horizontal)

    for x in horizontal:
        for y in vertical:
            hold = f"{x}{y}"
            # Group by hold and calculate mean if the hold exists
            try:
                if kpi == 'grade':
                    mean = df.groupby(hold).agg({'grade': 'mean'}).iloc[1].grade
                    heatmap_df.loc[y, x] = mean
                if kpi == 'count':
                    count = df.query(f"{hold} == 1").shape[0]
                    heatmap_df.loc[y, x] = count
            except:
                heatmap_df.loc[y, x] = np.NaN  

    # Reverse the order of the index (which represents the vertical holds)
    heatmap_df = heatmap_df.loc[::-1]

    # convert the values to numeric
    heatmap_df = heatmap_df.apply(pd.to_numeric)

    # Plot the heatmap
    plt.figure(figsize=(12, 8))
    ax = sns.heatmap(heatmap_df, cmap='coolwarm', cbar=True)
    if kpi == 'grade':
        plt.title('Average grade of boulder when hold is used')
    if kpi == 'count':
        plt.title('Number of routes that use this hold')
    plt.tight_layout()

    if kpi == 'grade':
        cbar = ax.collections[0].colorbar
        value_range = range(1, 12)
        translated_labels = [p.GRADE_DICT_REVERSE[value] for value in value_range]

        cbar.set_ticks(value_range)  # Set the tick positions
        cbar.set_ticklabels(translated_labels)  # Set the tick labels

    plt.show()


In [ ]:
get_holds_heatmap(df, kpi='grade')

In [ ]:
get_holds_heatmap(df, kpi='count')